## 1.  Construct a class to store/represent graph

In [1]:
class Graph(object):                            # a graph containing vertices and edges
  def __init__(self):
      self._vertices = []                       # a list/arrary of vertices
      self._adjacency_list = {}                 # a hash table mapping vertex pairs to 1

  def nVertices(self):                          # Get the number of graph vertices
      return len(self._vertices)                # return the length of the vertices list

  def nEdges(self):                             # Get the number of graph edges
      return len(self._adjacency_list) // 2     # divde the # of keys by 2

  def add_vertex(self, vertex):                 # add a new vertex to the graph
      self._vertices.append(vertex)             # place at end of vertex list
      # FIXED: Removed line that incorrectly added vertices to the edge map
      # self._adjacency_list[vertex] = []

  def validIndex(self, n):                      # check n is a valid vertex index
      if n < 0 or self.nVertices() <= n:        # if it lies outside the valid range,
          raise IndexError("Invalid Vertex Index") # raise an exception
      return True

  def getVertex(self, n):                      # return the vertex at index n
      if self.validIndex(n):                   # check that n is a valid vertex index
          return self._vertices[n]             # return nth vertex

  def add_edge(self, a, b):                    # add a new edge to the graph
      self.validIndex(a)                       # check a is a valid vertex index
      self.validIndex(b)                       # check b is a valid vertex index
      if a == b:                               # If vertices are the same
          raise ValueError("Vertices are the same") # Raise the exception
      self._adjacency_list[a,b] = 1            # add edge
      self._adjacency_list[b,a] = 1            # add edge the reserve direction

  def hasEdge(self, a, b):                     # check if an edge exists
      self.validIndex(a)                       # check a is a valid vertex index
      self.validIndex(b)                       # check b is a valid vertex index
      return self._adjacency_list.get((a,b), False)

  # --- implement for traversing vertices ---

  def vertices(self):                          # generate sequence of all vertex indices
      return range(self.nVertices())

  def adjacentVertices(self, n):               # generate sequence of all adjacent vertices
      self.validIndex(n)                       # check n is a valid vertex index
      for j in self.vertices():
          if j != n and self.hasEdge(n, j):    # if other vertex connects
              yield j                          # yield the adjacent vertex

  def adjacentUnvisitedVertices(self, n, visited, markVisits=True):     # generate sequence of all adjacent unvisited vertices
      for j in self.adjacentVertices(n):
          if not visited[j]:
              if markVisits:
                  visited[j] = True
              yield j

  # Minimum Spanning Tree
  def minimumSpanningTree(self, n):
      self.validIndex(n)               # Check that vertex n is valid
      tree = Graph()                   # Initial MST is an empty graph
      vMap = [None] * self.nVertices() # Array to map vertex indices
      for vertex, path in self.depthFirst(n):
         vMap[vertex] = tree.nVertices() # DF visited vertex will be
         tree.add_vertex(self.getVertex(vertex)) # last vertex in MST as we add it
         if len(path) > 1:  # If the path has more than one vertex,
            # add last edge in path to MST, mapping
            tree.add_edge(vMap[path[-2]], vMap[path[-1]])
      return tree

  # Summarize the graph in a string
  def __str__(self):
      nVertices = self.nVertices()
      nEdges = self.nEdges()
      return '<Graph of {} vert{} and {} edge{}>'.format(
         nVertices, 'ex' if nVertices == 1 else 'ices',
         nEdges, '' if nEdges == 1 else 's')

  # Pring all the graph's vertices and edges
  def print(self, prefix=''):                  # prefix each line with given string
      print('{}{}'.format(prefix, self))       # Print summary form of graph
      for vertex in self.vertices():           # Loop over all vertex indices
         print('{}{}:'.format(prefix, vertex), # Print vertex index
               self.getVertex(vertex))         # and string form of vertex
         for k in range(vertex + 1, self.nVertices()): # Loop over
             if self.hasEdge(vertex, k): # higher vertex indices, if
               print(prefix, # there's an edge to it, print edge
                     self._vertices[vertex].name,
                     '<->',
                     self._vertices[k].name)

  # --- implement depthFirst ----
  def depthFirst(self, n):
      self.validIndex(n)
      visisted = [False] * self.nVertices()
      stack = Stack ()
      stack.push(n)
      visisted[n] = True
      yield (n, list(stack))
      while not stack.isEmpty():
          visit = stack.peek()
          adj = None
          for j in self.adjacentUnvisitedVertices(visit, visisted):
              adj = j
              break
          if adj is None:
              stack.pop()
          else:
              stack.push(adj)
              yield (adj, list(stack))

  # --- implement BreadFirst ---

  def breadthFirst(self, n):                  # Traverse the vertices in breadth-first
      self.validIndex(n)                      # Order starting at vertex n
      visited = [False] * self.nVertices()    # Check that vertex n is valid
      queue = Queue()                         # start an empty queue
      queue.insert(n)                         # insert the starting vertex index
      visited[n] = True                       # and mark starting vertex as visited
      while not queue.isEmpty():              # Loop until nothing left on queue
          visit = queue.remove()              # Visit vertext at front of queu
          yield (visit)                       # yield vertex to visit it
          # loop over adjacent unvisited vertices
          for j in self.adjacentUnvisitedVertices(visit, visited, markVisits=False):
              visited[j] = True
              queue.insert(j)

  def display_graph(self):
        """Prints the vertices and the connections stored in the graph."""
        print("--- Graph Representation (Index-based Edges) ---")

        # 1. Display Vertices and their Indices
        print("Vertices:")
        for i, vertex_data in enumerate(self._vertices):
            print(f"  Index {i}: {vertex_data}")

        # 2. Display Edges
        print("\nEdges:")
        displayed_edges = set()
        for edge_tuple in self._adjacency_list.keys():
            u_index, v_index = edge_tuple

            # Use a set to only display the edge once (since it's undirected)
            if tuple(sorted(edge_tuple)) not in displayed_edges:
                u_data = self.getVertex(u_index)
                v_data = self.getVertex(v_index)
                print(f"  ({u_index}, {v_index}): {u_data} --- {v_data}")
                displayed_edges.add(tuple(sorted(edge_tuple)))

        if not displayed_edges:
            print("  No edges found.")
        print(f"\nTotal Vertices: {self.nVertices()}, Total Edges: {self.nEdges()}")

class Stack(list):                              # use list to define Stack class
  def push(self, item):
    self.append(item)
  def peek(self):
    return self[-1]               # last element is top of stack
  def isEmpty(self):
     return len(self) == 0     # check if stack is empty

class Queue(list):
  def insert(self, item):
    self.append(item)
  def remove(self):
    return self.pop(0)
  def isEmpty(self):
    return len(self) == 0
  def peek(self):
    return self[0]

class Vertex(object):
    """Simple class to hold vertex data with a 'name' attribute."""
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return f'Vertex({self.name})'
    def __repr__(self):
        return self.name

## Test and validate all functions in the graph as well as BFS and DFS algorithm

In [2]:
import math
from collections import deque

# --- TESTING FUNCTIONS ---

def setup_test_graph():
    """Sets up a graph with 8 vertices (A-H) and specific edges for traversal testing."""
    g = Graph()
    g.add_vertex(Vertex('A')) # Index 0
    g.add_vertex(Vertex('B')) # Index 1
    g.add_vertex(Vertex('C')) # Index 2
    g.add_vertex(Vertex('D')) # Index 3
    g.add_vertex(Vertex('E')) # Index 4
    g.add_vertex(Vertex('F')) # Index 5
    g.add_vertex(Vertex('G')) # Index 6
    g.add_vertex(Vertex('H')) # Index 7

    # Edges: A(0) <-> B(1), A(0) <-> C(2)
    g.add_edge(0, 1)
    g.add_edge(0, 2)
    # Edges: B(1) <-> D(3)
    g.add_edge(1, 3)
    # Edges: C(2) <-> E(4)
    g.add_edge(2, 4)
    # Edges: D(3) <-> F(5)
    g.add_edge(3, 5)
    # Edges: E(4) <-> G(6)
    g.add_edge(4, 6)
    # Edge: F(5) <-> H(7)
    g.add_edge(5, 7)
    # Edge: G(6) <-> H(7)
    g.add_edge(6, 7)
    return g

def run_basic_tests(g):
    """Tests basic methods: nVertices, nEdges, add_vertex, add_edge, hasEdge, validIndex."""
    print("## ✅ Basic Property and Edge Tests")
    print("---")

    # Test property methods
    assert g.nVertices() == 8, f"nVertices failed: Expected 8, got {g.nVertices()}"
    print(f"nVertices: {g.nVertices()}")

    # Count the number of edges: 8
    # (0,1), (0,2), (1,3), (2,4), (3,5), (4,6), (5,7), (6,7)
    assert g.nEdges() == 8, f"nEdges failed: Expected 8, got {g.nEdges()}"
    print(f"nEdges: {g.nEdges()}")

    # Test edge methods
    assert g.hasEdge(0, 1) == 1, "hasEdge(0, 1) failed: Expected 1"
    assert g.hasEdge(1, 0) == 1, "hasEdge(1, 0) failed: Expected 1 (Undirected)"
    assert g.hasEdge(0, 3) == False, "hasEdge(0, 3) failed: Expected False (No direct edge)"
    print("hasEdge checks passed.")

    # Test error handling
    try:
        g.add_edge(1, 1)
        assert False, "add_edge loop check failed: Should raise ValueError"
    except ValueError:
        print("add_edge(1, 1) correctly raised ValueError.")

    try:
        g.validIndex(8)
        assert False, "validIndex failed: Should raise IndexError for index 8"
    except IndexError:
        print("validIndex(8) correctly raised IndexError.")

    print("\nBasic tests concluded successfully.")

def run_traversal_tests(g):
    """Tests DFS and BFS algorithms."""
    print("\n## 🧭 Traversal Algorithm Tests")
    print("---")

    # --- Breadth-First Search (BFS) Test ---
    # Expected order starting from A(0):
    # Level 0: A(0)
    # Level 1: B(1), C(2)
    # Level 2: D(3), E(4)
    # Level 3: F(5), G(6)
    # Level 4: H(7)
    expected_bfs = [0, 1, 2, 3, 4, 5, 6, 7]
    actual_bfs = list(g.breadthFirst(0))

    print(f"BFS Order (Indices): {actual_bfs}")
    assert actual_bfs == expected_bfs, "BFS Failed: Order is incorrect"
    print("BFS Test Passed.")
    print(f"BFS Order (Names): {' '.join(g.getVertex(i).name for i in actual_bfs)}")

    # --- Depth-First Search (DFS) Test ---
    # Expected path starting from A(0) (following smallest index first):
    # The depthFirst implementation, when exploring neighbors in ascending index order,
    # will produce the following sequence:
    # A(0) -> B(1) -> D(3) -> F(5) -> H(7) -> G(6) -> E(4) -> C(2)
    expected_dfs_indices = [0, 1, 3, 5, 7, 6, 4, 2]

    # The yield structure of depthFirst is (vertex_index, path_stack)
    actual_dfs = [index for index, path in g.depthFirst(0)]

    print(f"\nDFS Order (Indices): {actual_dfs}")
    assert actual_dfs == expected_dfs_indices, "DFS Failed: Order is incorrect"
    print("DFS Test Passed.")
    print(f"DFS Order (Names): {' '.join(g.getVertex(i).name for i in actual_dfs)}")

    print("\nTraversal tests concluded successfully.")

# --- EXECUTION ---

if __name__ == "__main__":
    print("Starting Graph Class Test Suite...\n")

    # 1. Setup the Graph
    test_graph = setup_test_graph()
    test_graph.display_graph()

    # 2. Run Basic Tests
    run_basic_tests(test_graph)

    print("\n" + "="*50)

    # 3. Run Traversal Tests
    # Note: BFS method was modified in the code above to correctly update 'visited'
    run_traversal_tests(test_graph)

    print("\n" + "="*50)
    print("✨ All tests passed! The Graph class methods are functioning as intended.")

Starting Graph Class Test Suite...

--- Graph Representation (Index-based Edges) ---
Vertices:
  Index 0: Vertex(A)
  Index 1: Vertex(B)
  Index 2: Vertex(C)
  Index 3: Vertex(D)
  Index 4: Vertex(E)
  Index 5: Vertex(F)
  Index 6: Vertex(G)
  Index 7: Vertex(H)

Edges:
  (0, 1): Vertex(A) --- Vertex(B)
  (0, 2): Vertex(A) --- Vertex(C)
  (1, 3): Vertex(B) --- Vertex(D)
  (2, 4): Vertex(C) --- Vertex(E)
  (3, 5): Vertex(D) --- Vertex(F)
  (4, 6): Vertex(E) --- Vertex(G)
  (5, 7): Vertex(F) --- Vertex(H)
  (6, 7): Vertex(G) --- Vertex(H)

Total Vertices: 8, Total Edges: 8
## ✅ Basic Property and Edge Tests
---
nVertices: 8
nEdges: 8
hasEdge checks passed.
add_edge(1, 1) correctly raised ValueError.
validIndex(8) correctly raised IndexError.

Basic tests concluded successfully.


## 🧭 Traversal Algorithm Tests
---
BFS Order (Indices): [0, 1, 2, 3, 4, 5, 6, 7]
BFS Test Passed.
BFS Order (Names): A B C D E F G H

DFS Order (Indices): [0, 1, 3, 5, 7, 6, 4, 2]
DFS Test Passed.
DFS Order (Nam

## 2. The Implementation of Vertex k-labeling for Perfect Ternary Tree (**T**m,h) m = 3 for Ternary tree and H for the height of the tree



In [3]:
import math
from collections import deque

class PerfectTernaryTree:
    def __init__(self, height):
        self._m = 3                           # Ternary tree factor
        self._h = height

        # Topology: caculate vertices adn initilize the storage
        self._V = self._calculate_total_vertices()
        self._E = self._V - 1

        # Initialize an array/list of size V+1 (uisng 1-based indexing)
        self._labels = [0] * (self._V + 1)

        # 2. Optimization Target: K approx V/2
        self._k_max = math.ceil(self._V / 2)

    def _calculate_total_vertices(self):
        return (self._m**(self._h + 1) - 1) // (self._m - 1)

    def nVertices(self):
        return self._V

    # --- O(1) Structure Functions (Implicit Topology) ---
    def get_parent(self, i):
        if i == 1: return None
        return (i - 2) // self._m + 1

    def get_children(self, i):
        """Returns the indices of the m children of node i in O(1) time (1-indexed)."""
        # For a 1-indexed perfect m-ary tree:
        # First child of node i is m*(i-1) + 2
        # Last child of node i is m*(i-1) + 2 + (m-1) which simplifies to m*i + 1
        first = self._m * (i - 1) + 2
        last = self._m * i + 1

        children_indices = []
        for child_idx in range(first, last + 1):
            if child_idx <= self._V:
                children_indices.append(child_idx)
        return children_indices

    def _calculate_dynamic_root_weights(self):
        """
        Calculates root weights based on the K-Max Constraint.
        Constraint: The Level 1 Node Labels must not exceed K_max.
        Since Parent=1, Weight must be <= K_max + 1.
        """
        # 1. The Rightmost weight is the ceiling (K_max + 1)
        #    This ensures Node 4 gets exactly Label = K_max.
        w_right = self._k_max + 1

        # 2. The Leftmost weight is the floor.
        w_left = 2

        # 3. The Middle weight is the geometric or arithmetic center.
        #    Arithmetic mean works best for uniform distribution.
        w_mid = (w_left + w_right) // 2

        return [w_left, w_mid, w_right]

    def label_tree(self):
        """ Implements the generalized m-ary labeling strategy O(V) complexity)."""
        if self._V == 0:
            return

        print(f"\n=== Processing Height {self._h} (V={self._V}, E={self._E}) ===")
        print(f"Target Max Label (K): {self._k_max}")

        self._labels[1] = 1                     # root label = 1
        root_children = self.get_children(1)

        # 1. CALCULATE CORRECT ROOT WEIGHTS
        root_edge_weights = self._calculate_dynamic_root_weights()
        print(f"Calculated Root Weights: {root_edge_weights}")

        # Apply Root Weights
        for i, child_idx in enumerate(root_children):
            weight = root_edge_weights[i]
            self._labels[child_idx] = weight - self._labels[1]

        # 2. GENERATE REMAINING POOL
        # Distribute ALL other weights (up to E+1)
        all_weights = set(range(2, self._E + 2))
        for w in root_edge_weights:
            if w in all_weights:
                all_weights.remove(w)

        sorted_weights = sorted(list(all_weights))

        # 3. PARTITION POOLS
        # We apply the partitioning strategy by spliting  the remaining weights into 3 equal chunks.
        # Even though Root weights are "compressed" into the lower half,
        # the Right subtree is capable of handling the largest weights because
        # its Root Label is high (K_max).
        chunk_size = len(sorted_weights) // 3
        remainder = len(sorted_weights) % 3

        size_1 = chunk_size + (1 if remainder > 0 else 0)
        size_2 = chunk_size + (1 if remainder > 1 else 0)

        left_pool = deque(sorted_weights[:size_1])
        mid_pool = deque(sorted_weights[size_1 : size_1 + size_2])

        # Right pool gets the largest weights and is REVERSED (Descending)
        right_pool = deque(sorted(sorted_weights[size_1 + size_2:], reverse=True))

        print(f"Partition Sizes: L={len(left_pool)}, M={len(mid_pool)}, R={len(right_pool)}")

        # 4. RECURSIVE ASSIGNMENT
        self._recursive_label(root_children[0], left_pool, "Left")
        self._recursive_label(root_children[1], mid_pool, "Mid")
        self._recursive_label(root_children[2], right_pool, "Right")

        return self._labels[1:]

    def _recursive_label(self, root_index, weight_queue, name):
        """ Recursive core. Labels the children of root_index and traverses."""
        children = self.get_children(root_index)
        if not children:                          # base case: leaf node
            return

        parent_label = self._labels[root_index]

        for child_index in children:
            if not weight_queue:
                return

            # Retry Logic: Rotate queue to find valid label
            attempts = 0
            max_attempts = len(weight_queue)
            found = False

            while attempts < max_attempts:
                edge_weight = weight_queue.popleft()
                k_label = edge_weight - parent_label

                if 0 < k_label <= self._k_max:
                    self._labels[child_index] = k_label
                    found = True
                    break
                else:
                    weight_queue.append(edge_weight)
                    attempts += 1

            if not found:
                # Critical Failure reporting
                print(f"  [CRITICAL FAIL] {name} Subtree: Node {child_index} (Parent L={parent_label}). "
                      f"No valid weight in queue satisfying 0 < L <= {self._k_max}")
                return

            self._recursive_label(child_index, weight_queue, name)

    # iv. Store the lables of vertices and weights of the edges
    def get_outcome(self):
        """
        Returns the structured results of the labeling.
        Returns:s
            A list of dictionaries, where each item represents a node/edge.
        """
        outcome = []
        # Node 1 is Root, has no parent edge
        outcome.append({
            "node_id": 1,
            "parent_id": None,
            "edge_weight": None,
            "vertex_label": self._labels[1]
        })

        for i in range(2, self._V + 1):
            parent = (i - 2) // self._m + 1
            # Calculate edge_weight from stored labels
            edge_weight = self._labels[i] + self._labels[parent]
            outcome.append({
                "node_id": i,
                "parent_id": parent,
                "edge_weight": edge_weight, # Use the calculated edge_weight
                "vertex_label": self._labels[i]
            })
        return outcome

    # v. Compare the outcome with mathematical property and tabulate
    def compare_results_with_theory(self): # Changed signature
        # --- 1. THEORETICAL CALCULATIONS ---
        m = 3
        # Formula: (m^(h+1) - 1) / (m - 1)
        theo_V = (m**(self._h + 1) - 1) // (m - 1) # Use self._h
        theo_E = theo_V - 1
        theo_K_max = math.ceil(self._V / 2) # Use self._V
        # The set of expected weights is {2, 3, ..., E+1}
        theo_weight_sum = sum(range(2, theo_E + 2))

        # --- 2. ACTUAL RESULTS (FROM CODE) ---
        results = self.get_outcome() # Use self.get_outcome()

        act_V = len(results)
        # Count valid edges (exclude Root which has no parent edge)
        edges = [r['edge_weight'] for r in results if r['edge_weight'] is not None]
        act_E = len(edges)

        # Get max label used
        labels = [r['vertex_label'] for r in results]
        act_max_label = max(labels) if labels else 0

        act_weight_sum = sum(edges)

        # Check for uniqueness of weights
        unique_weights = len(set(edges))
        is_bijective = (unique_weights == act_E) and (unique_weights == theo_E)

        # --- 3. TABULATE OUTCOME ---
        print(f"\n{'METRIC':<25} | {'THEORY (Math)':<20} | {'ACTUAL (Algo)':<20} | {'STATUS':<10}")
        print("-" * 85)

        # Helper to print rows
        def print_row(metric, theo, act):
            status = "PASS" if theo == act else "FAIL"
            print(f"{metric:<25} | {str(theo):<20} | {str(act):<20} | {status:<10}")

        print_row("Total Vertices (V)", theo_V, act_V)
        print_row("Total Edges (E)", theo_E, act_E)
        print_row("Max Label Constraint (K)", theo_K_max, self._k_max) # Use self._k_max

        # For Max Label, Actual must be <= Theory, not necessarily equal
        lbl_status = "PASS" if act_max_label <= theo_K_max else "FAIL"
        print(f"{'Max Label Used':<25} | {'<= ' + str(theo_K_max):<20} | {str(act_max_label):<20} | {lbl_status:<10}")

        print_row("Unique Edge Weights", theo_E, unique_weights)
        print_row("Sum of Edge Weights", theo_weight_sum, act_weight_sum)

        print("-" * 85)
        print(f"Weight Bijectivity Check: {'SUCCESS' if is_bijective else 'FAILURE'}")
        print(f"  - Theoretical Range: [2 ... {theo_E + 1}]")
        print(f"  - Actual Range:      [{min(edges)} ... {max(edges)}]")

    def verify_tree_properties(self):
        """Checks if all edge weights are unique and labels are within bounds."""
        edge_weights = set()
        max_k_used = 0
        min_k_used = float('inf')

        print("\n--- Verification Report ---")

        for i in range(1, self._V + 1):
            # if self._labels[i] > max_k_used: max_k_used = self._labels[i]
            # if self._labels[i] < min_k_used: min_k_used = self._labels[i]
            parent_index = self.get_parent(i)
            max_k_used = max(max_k_used, self._labels[i])
            min_k_used = min(min_k_used, self._labels[i])

            if parent_index is not None:
                weight = self._labels[i] + self._labels[parent_index]

                # Check 1: Range
                if weight < 2 or weight > self._E + 1:
                    print(f"FAIL: Weight {weight} out of range (2-{self._E + 1})")
                    return False

                # Check 2: Duplicates
                if weight in edge_weights:
                    print(f"FAIL: Duplicate weight {weight} found at Node {i}")
                    return False
                edge_weights.add(weight)

        # Check 3: Completeness
        if len(edge_weights) != self._E:
             print(f"FAIL: Only {len(edge_weights)} unique weights found. Expected {self._E}.")
             return False

        # Check 4: K-Label Bound
        if max_k_used > self._k_max:
             print(f"FAIL: Max Label {max_k_used} exceeds limit {self._k_max}!")
             return False

        if min_k_used < 1:
             print(f"FAIL: Label {min_k_used} is non-positive!")
             return False

        print(f"SUCCESS: All {self._E} edges have unique weights (2-{self._E + 1}).")
        print(f"Labels satisfy range [{min_k_used}...{max_k_used}] <= {self._k_max}")
        print(f"Max Label Used: {max_k_used}")
        print(f"Expected Max K Lable: {self._k_max}")
        return True

## Testing Perfect Ternary Tree

In [4]:
def run_experiment():
    # --- Set Height ---
    height = 5
    tree = PerfectTernaryTree(height)

    # Run the labeling algorithm
    final_labels = tree.label_tree()

    # Display the result
    print("\n--- Vertex K-Labels ---")
    print(f"Root (1): {final_labels[0]}")
    print(f"L1 (Nodes 2-4): {final_labels[1:4]}")
    print(f"L2 Left (Children of 2): {final_labels[4:7]}")
    print(f"L2 Mid  (Children of 3): {final_labels[7:10]}")
    print(f"L2 Right (Children of 4): {final_labels[10:13]}")

    # Verify the results
    is_valid = tree.verify_tree_properties()

    if is_valid:
        # Get the structured outcome
        results = tree.get_outcome()
        total_nodes = len(results)

        print(f"\n{'Node':<6} | {'Label':<6} | {'Parent':<6} | {'Par.Lbl':<8} | {'Edge Weight (u+v)':<18}")
        print("-" * 60)

        # Helper to print a row
        def print_row(row, all_results):
            node = row['node_id']
            label = row['vertex_label']
            parent_id = row['parent_id']
            parent_label = "-"
            if parent_id is not None:
                # Look up parent's label from all_results list
                # Assuming node_id is 1-indexed, so list index is node_id - 1
                parent_label = all_results[parent_id - 1]['vertex_label']

            weight = row['edge_weight'] if row['edge_weight'] is not None else "-"
            print(f"{node:<6} | {label:<6} | {parent_id if parent_id is not None else '-':<6} | {parent_label:<8} | {weight:<18}")

        # Print first 15 nodes
        for i in range(min(15, total_nodes)):
            print_row(results[i], results)

        if total_nodes > 30:
            print(f"... (Skipping {total_nodes - 30} intermediate nodes) ...")

        # Print last 15 nodes
        for i in range(max(15, total_nodes - 15), total_nodes):
            print_row(results[i], results)

        return results

    else:
        print("Tree labeling failed verification.")
        return None


#3. How the traversing will be appied?

The traversal strategy applied is Depth-First Search (DFS), specifically a Pre-order Traversal by diving deep into a branch before moving to the next subtree

1.   Visit Parent: start at a node (e.g., Node 2).
2.   Process Edge: pop a weight from the queue and assign the label for the first child (e.g., Node 5).
3.   Recurse (Dive):  immediately call `_recursive_label` on Node 5.
4.   Repeat: The code will go all the way to the bottom leaf (Node 14, 15, etc.) before it comes back up to finish Node 2's other children (Node 6, Node 7).

We are using partitioning strategy because DFS concentrate the specific weights in their specific regions.












#4. Store the labels of vertices and weights of the edges as an outcome

In [5]:
import pandas as pd

# Run the experiment and get the results
results = run_experiment()

if results is not None:
    # save to dataframe and csv file and display
    df = pd.DataFrame(results)
    print("\n--- Outcome DataFrame (First 5 Rows) ---")
    display(df.head())
    print("\n-- Outcome DataFrame (Last 5 Rows) --")
    display(df.tail())
    df.to_csv('output.csv', index=False)
    print("\nDataFrame saved to 'output.csv'")
else:
    print("No results to display or save due to failed verification.")


=== Processing Height 5 (V=364, E=363) ===
Target Max Label (K): 182
Calculated Root Weights: [2, 92, 183]
Partition Sizes: L=120, M=120, R=120

--- Vertex K-Labels ---
Root (1): 1
L1 (Nodes 2-4): [1, 91, 182]
L2 Left (Children of 2): [2, 42, 82]
L2 Mid  (Children of 3): [33, 73, 114]
L2 Right (Children of 4): [182, 142, 102]

--- Verification Report ---
SUCCESS: All 363 edges have unique weights (2-364).
Labels satisfy range [1...182] <= 182
Max Label Used: 182
Expected Max K Lable: 182

Node   | Label  | Parent | Par.Lbl  | Edge Weight (u+v) 
------------------------------------------------------------
1      | 1      | -      | -        | -                 
2      | 1      | 1      | 1        | 2                 
3      | 91     | 1      | 1        | 92                
4      | 182    | 1      | 1        | 183               
5      | 2      | 2      | 1        | 3                 
6      | 42     | 2      | 1        | 43                
7      | 82     | 2      | 1        | 83     

,node_id,parent_id,edge_weight,vertex_label
0,1,NaN,NaN,1
1,2,1.0,2.0,1
2,3,1.0,92.0,91
3,4,1.0,183.0,182
4,5,2.0,3.0,2



-- Outcome DataFrame (Last 5 Rows) --


,node_id,parent_id,edge_weight,vertex_label
359,360,120.0,250.0,153
360,361,120.0,249.0,152
361,362,121.0,247.0,154
362,363,121.0,246.0,153
363,364,121.0,245.0,152



DataFrame saved to 'output.csv'


# 5. Compare your results with mathematical property and tabulate the outcomes for comparison.

The Mathematical Properties (Theory)

To build the "Theoretical" column of your comparison, we use the following definitions for a Perfect Ternary Tree of height $h$:Total Vertices ($V$): The sum of a geometric series.$$V = \sum_{i=0}^{h} m^i = \frac{m^{h+1} - 1}{m - 1}$$Total Edges ($E$): In any tree, the number of edges is vertices minus one.$$E = V - 1$$Optimization Constraint ($K_{max}$): Your specific algorithm constrains labels to half the vertex count.$$K_{max} = \lceil \frac{V}{2} \rceil$$Target Weight Set ($W$): The algorithm targets a "consecutive" or "rainbow" edge weight distribution starting at 2 (since minimal labels are $1+1$).$$W = \{2, 3, \dots, E+1\}$$

In [6]:
h = 5
my_tree = PerfectTernaryTree(h)
my_tree.label_tree()
my_tree.compare_results_with_theory()


=== Processing Height 5 (V=364, E=363) ===
Target Max Label (K): 182
Calculated Root Weights: [2, 92, 183]
Partition Sizes: L=120, M=120, R=120

METRIC                    | THEORY (Math)        | ACTUAL (Algo)        | STATUS    
-------------------------------------------------------------------------------------
Total Vertices (V)        | 364                  | 364                  | PASS      
Total Edges (E)           | 363                  | 363                  | PASS      
Max Label Constraint (K)  | 182                  | 182                  | PASS      
Max Label Used            | <= 182               | 182                  | PASS      
Unique Edge Weights       | 363                  | 363                  | PASS      
Sum of Edge Weights       | 66429                | 66429                | PASS      
-------------------------------------------------------------------------------------
Weight Bijectivity Check: SUCCESS
  - Theoretical Range: [2 ... 364]
  - Actual Range: 